<a href="https://colab.research.google.com/github/vincetofy223/federated-privacy-ml-paper/blob/main/federated_learning_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q flwr==1.9.0 tensorflow==2.19.0 matplotlib numpy==1.26.4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.7/364.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.0/645.0 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 108.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.1/250.1 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.49.1 requires typer<1.0,>=0.12, but you have typer 0.9.4 which is incompatible.


In [2]:
import flwr as fl
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

print("✅ Libraries imported successfully.")
print("TensorFlow version:", tf.__version__)


✅ Libraries imported successfully.
TensorFlow version: 2.19.0


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [3]:
# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize the pixel values (0–255 → 0–1)
x_train, x_test = x_train / 255.0, x_test / 255.0

# Split training data into 5 clients (for simulation)
NUM_CLIENTS = 5
client_data = []
split_size = len(x_train) // NUM_CLIENTS

for i in range(NUM_CLIENTS):
    start = i * split_size
    end = start + split_size
    client_data.append((x_train[start:end], y_train[start:end]))

print(f"✅ Data prepared for {NUM_CLIENTS} clients.")
print("Each client has:", len(client_data[0][0]), "samples")


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
✅ Data prepared for 5 clients.
Each client has: 12000 samples
